# What Should I Eat? - Recipe Suggester Agent

## Description

Look, I love cooking, but the daily "what's for dinner" question drives me crazy. I'll stand in front of my open fridge for like 10 minutes, staring at random ingredients, trying to remember if I can even eat half of them (shellfish allergy, hate mushrooms). Then I spend another 15 minutes scrolling through recipe sites that don't match what I actually have at home.

So I built this agent to solve my own problem. You just tell it what's in your fridge - eggs, cheese, some sad tomatoes, whatever - and it suggests 3 recipes you can actually make right now. The cool part is it remembers your dietary restrictions. I'm pescatarian and allergic to shellfish, so it won't suggest shrimp stir-fry even if I have all the ingredients.

The agent uses Gemini to understand what you're working with and come up with ideas that make sense. It's not just throwing random recipes at you - it actually considers what goes together and what you can realistically cook. Plus it finds recipe links for you, so you don't have to Google "chicken bell pepper recipe" and scroll through 50 blog posts about someone's vacation before getting to the actual instructions.

It's simple but it works. Saves me probably 20-30 minutes every day and I waste way less food now. Been using it for quick lunches and weeknight dinners mostly.

## Architecture & Features

Basic setup: you input ingredients → Gemini processes with your saved preferences (allergies, dislikes, diet) → outputs 3 recipe suggestions → optional recipe link lookup.

Three main components working together:
- **LLM Agent (Gemini 2.0)**: Does the actual recipe thinking
- **Memory**: Stores your food preferences so you don't repeat yourself
- **Session management**: Keeps conversation context if you want multiple suggestions

Also added a recipe link finder that searches for actual URLs. Works pretty well most of the time, if not you will be redirected to a simple google research of your recipe.

## Tech
Gemini 2.0 Flash API, Python, simple in-memory storage for the demo.

In [1]:
# What Should I Eat? Agent
# Simple recipe suggester based on fridge ingredients

import os
import requests
from google import genai
from google.genai import types
from kaggle_secrets import UserSecretsClient

# Setup API
user_secrets = UserSecretsClient()
api_key = user_secrets.get_secret("GOOGLE_API_KEY")
client = genai.Client(api_key=api_key)

# User preferences storage
user_prefs = {
    "allergies": [],
    "dislikes": [],
    "diet": None
}

# Conversation history
chat_history = []

def get_recipe_suggestions(ingredients, prefs):
    """Main function to get recipe ideas from Gemini"""
    
    # Build the prompt with user context
    context = "You are a helpful and flexible cooking assistant.\n"
    
    if prefs["allergies"] and prefs["allergies"][0].lower() not in ['no', 'none', '']:
        context += f"IMPORTANT - User is allergic to: {', '.join(prefs['allergies'])}\n"
    
    if prefs["dislikes"] and prefs["dislikes"][0].lower() not in ['no', 'none', '']:
        context += f"User dislikes: {', '.join(prefs['dislikes'])}\n"
    
    if prefs["diet"] and prefs["diet"].lower() not in ['no', 'none', '']:
        context += f"User prefers: {prefs['diet']} diet\n"
        context += f"IMPORTANT: If user provides non-{prefs['diet']} ingredients, IGNORE them and suggest recipes using only the {prefs['diet']}-compatible ingredients.\n"
    
    context += "\nBe practical and helpful. Work with what the user has.\n"
    
    prompt = f"I have these ingredients: {', '.join(ingredients)}\n\nSuggest 3 simple recipes I can make with the compatible ingredients."
    
    # Call Gemini
    response = client.models.generate_content(
        model="gemini-2.0-flash-exp",
        contents=prompt,
        config=types.GenerateContentConfig(
            system_instruction=context,
            temperature=0.8,
            max_output_tokens=800
        )
    )
    
    return response.text

def check_url_exists(url):
    """Check if a URL actually exists"""
    try:
        response = requests.head(url, timeout=5, allow_redirects=True)
        return response.status_code < 400
    except:
        return False

def find_recipe_url(recipe_name):
    """Try to find a real recipe URL, fallback to Google search"""
    
    # Common recipe sites
    sites = [
        "allrecipes.com",
        "simplyrecipes.com", 
        "foodnetwork.com",
        "delish.com",
        "bonappetit.com"
    ]
    
    # Try to construct likely URLs
    clean_name = recipe_name.lower().replace(' ', '-').replace(',', '')
    
    for site in sites:
        test_url = f"https://www.{site}/recipe/{clean_name}"
        if check_url_exists(test_url):
            return test_url
    
    # If no direct URL found, return Google search
    search_query = recipe_name.replace(' ', '+').replace(',', '')
    return f"https://www.google.com/search?q={search_query}+recipe"

# Demo run
print("=" * 70)
print("WHAT SHOULD I EAT? - Recipe Agent Demo")
print("=" * 70)
print()

# Set some example preferences
user_prefs["allergies"] = ["shellfish"]
user_prefs["dislikes"] = ["mushrooms"] 
user_prefs["diet"] = "pescatarian"

print("Your profile:")
print(f"  Allergies: {', '.join(user_prefs['allergies'])}")
print(f"  Dislikes: {', '.join(user_prefs['dislikes'])}")
print(f"  Diet: {user_prefs['diet']}")
print()

# Example 1
print("-" * 70)
print("Scenario 1: Quick lunch")
print("-" * 70)
ingredients1 = ["eggs", "cheese", "bread", "tomatoes", "spinach"]
print(f"Ingredients: {', '.join(ingredients1)}")
print()

recipes1 = get_recipe_suggestions(ingredients1, user_prefs)
print(recipes1)
print()

# Get link for first recipe
print("Finding recipe links...")
lines = recipes1.split('\n')
recipe_count = 0
for line in lines:
    if line.strip() and any(c.isdigit() for c in line[:3]):
        # Parse recipe name
        clean = line.replace('**', '').replace('*', '').strip()
        import re
        clean = re.sub(r'^\d+[\.\)]\s*', '', clean)
        
        if ':' in clean:
            recipe_name = clean.split(':')[0].strip()
        else:
            recipe_name = clean.split('.')[0].strip() if '.' in clean else clean.strip()
        
        if len(recipe_name) > 5 and len(recipe_name) < 70:
            link = find_recipe_url(recipe_name)
            if "google.com/search" in link:
                print(f"  → {recipe_name}: Google Search → {link}")
            else:
                print(f"  → {recipe_name}: Found! → {link}")
            recipe_count += 1
            if recipe_count >= 3:
                break

print()

# Example 2
print("-" * 70)
print("Scenario 2: Dinner time")
print("-" * 70)
ingredients2 = ["chicken", "rice", "bell peppers", "onions", "garlic", "soy sauce"]
print(f"Ingredients: {', '.join(ingredients2)}")
print()

recipes2 = get_recipe_suggestions(ingredients2, user_prefs)
print(recipes2)
print()

# Get links for scenario 2
print("Finding recipe links...")
lines = recipes2.split('\n')
recipe_count = 0
for line in lines:
    if line.strip() and any(c.isdigit() for c in line[:3]):
        clean = line.replace('**', '').replace('*', '').strip()
        import re
        clean = re.sub(r'^\d+[\.\)]\s*', '', clean)
        
        if ':' in clean:
            recipe_name = clean.split(':')[0].strip()
        else:
            recipe_name = clean.split('.')[0].strip() if '.' in clean else clean.strip()
        
        if len(recipe_name) > 5 and len(recipe_name) < 70:
            link = find_recipe_url(recipe_name)
            if "google.com/search" in link:
                print(f"  → {recipe_name}: Google Search → {link}")
            else:
                print(f"  → {recipe_name}: Found! → {link}")
            recipe_count += 1
            if recipe_count >= 3:
                break

print()
print("=" * 70)
print(f"Total queries: 2")
print("Demo complete!")
print("=" * 70)

WHAT SHOULD I EAT? - Recipe Agent Demo

Your profile:
  Allergies: shellfish
  Dislikes: mushrooms
  Diet: pescatarian

----------------------------------------------------------------------
Scenario 1: Quick lunch
----------------------------------------------------------------------
Ingredients: eggs, cheese, bread, tomatoes, spinach

Okay, here are 3 simple pescatarian recipes you can make with eggs, cheese, bread, tomatoes, and spinach:

1.  **Spinach and Tomato Omelet with Toast:**

    *   **Description:** A quick and easy omelet filled with fresh spinach and juicy tomatoes, served with a side of toast.
    *   **Instructions:**
        1.  Whisk 2-3 eggs with a pinch of salt and pepper.
        2.  Heat a small amount of oil or butter in a non-stick pan over medium heat.
        3.  Pour the egg mixture into the pan.
        4.  As the edges set, add a handful of spinach and some chopped tomatoes to one half of the omelet.
        5.  Sprinkle with cheese.
        6.  Fold the o

In [ ]:
# Interactive mode - try it yourself

print("\n" + "=" * 70)
print("INTERACTIVE MODE - Enter your own ingredients!")
print("=" * 70)
print()

# Reset for new user
my_prefs = {"allergies": [], "dislikes": [], "diet": None}

# Quick setup
print("Quick setup (press Enter to skip any):")
allergies_input = input("Any allergies? (comma separated): ").strip()
if allergies_input and allergies_input.lower() not in ['no', 'none']:
    my_prefs["allergies"] = [x.strip() for x in allergies_input.split(',')]

dislikes_input = input("Foods you don't like? (comma separated): ").strip()
if dislikes_input and dislikes_input.lower() not in ['no', 'none']:
    my_prefs["dislikes"] = [x.strip() for x in dislikes_input.split(',')]

diet_input = input("Diet type (vegetarian/vegan/etc)? ").strip()
if diet_input and diet_input.lower() not in ['no', 'none']:
    my_prefs["diet"] = diet_input

print()
if any([my_prefs["allergies"], my_prefs["dislikes"], my_prefs["diet"]]):
    print("Got it! Your preferences:")
    if my_prefs["allergies"]:
        print(f"  - Allergies: {', '.join(my_prefs['allergies'])}")
    if my_prefs["dislikes"]:
        print(f"  - Dislikes: {', '.join(my_prefs['dislikes'])}")
    if my_prefs["diet"]:
        print(f"  - Diet: {my_prefs['diet']}")
    print()

# Main loop
print("Now let's find you some recipes!")
print("(Type 'done' or 'quit' when finished)")
print()

while True:
    user_ingredients = input("What's in your fridge? (comma separated): ").strip()
    
    if user_ingredients.lower() in ['done', 'quit', 'exit', 'q']:
        print("\nHappy cooking! 👨‍🍳")
        break
    
    if not user_ingredients:
        continue
    
    # Parse ingredients
    ing_list = [x.strip() for x in user_ingredients.split(',') if x.strip()]
    
    if len(ing_list) == 0:
        print("Hmm, didn't catch that. Try again?")
        continue
    
    print(f"\nOk, searching recipes with: {', '.join(ing_list)}")
    print()
    
    # Get suggestions
    try:
        suggestions = get_recipe_suggestions(ing_list, my_prefs)
        print(suggestions)
        print()
        
        # Ask if they want a recipe link
        want_link = input("Want a link to one of these recipes? (recipe name or Enter to skip): ").strip()
        
        if want_link:
            print("Searching...")
            url = find_recipe_url(want_link)  # ← FIX ICI (c'était search_recipe_link avant)
            if "google.com/search" in url:
                print(f"Google Search: {url}")
            else:
                print(f"Found recipe: {url}")
        
    except Exception as e:
        print(f"Oops, something went wrong: {e}")
        print("Try again with different ingredients?")
    
    print("\n" + "-" * 70 + "\n")

print("\nThanks for trying the recipe agent!")


INTERACTIVE MODE - Enter your own ingredients!

Quick setup (press Enter to skip any):


Any allergies? (comma separated):  milk, apple
Foods you don't like? (comma separated):  pasta
Diet type (vegetarian/vegan/etc)?  Vegan



Got it! Your preferences:
  - Allergies: milk, apple
  - Dislikes: pasta
  - Diet: Vegan

Now let's find you some recipes!
(Type 'done' or 'quit' when finished)



What's in your fridge? (comma separated):  tomatoes, pasta, eggs, cheese, chicken, spinach, garlic, onions, milk, apple, eggs, pasta



Ok, searching recipes with: tomatoes, pasta, eggs, cheese, chicken, spinach, garlic, onions, milk, apple, eggs, pasta

Okay, I understand you need vegan recipes that avoid milk, apple, and pasta. Given the ingredients you have listed (tomatoes, spinach, garlic, and onions), here are three simple vegan recipes you can make:

**1. Sautéed Spinach and Tomatoes with Garlic & Onions**

*   **Ingredients:**
    *   Tomatoes
    *   Spinach
    *   Garlic
    *   Onions
    *   Olive oil (or other cooking oil)
    *   Salt and pepper to taste

*   **Instructions:**

    1.  Wash the spinach thoroughly.
    2.  Dice the tomatoes and onions. Mince the garlic.
    3.  Heat olive oil in a large pan or skillet over medium heat.
    4.  Add the onions and cook until softened, about 5 minutes.
    5.  Add the garlic and cook for another minute until fragrant.
    6.  Add the diced tomatoes and cook until they begin to break down and release their juices, about 8-10 minutes.
    7.  Add the spinach 

Want a link to one of these recipes? (recipe name or Enter to skip):  Simple Tomato Salad with Garlic and Onions


Searching...
Google Search: https://www.google.com/search?q=Simple+Tomato+Salad+with+Garlic+and+Onions+recipe

----------------------------------------------------------------------

